In [62]:
%config InlineBackend.figure_format = 'retina'

In [63]:
import copy
import json
import math
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as nptp
import pathlib
import prettytable
import threading
import typing as tp
import warnings

from dataclasses import dataclass, field
from mpl_toolkits.mplot3d import Axes3D
from sys import setrecursionlimit

setrecursionlimit(10 ** 9)
threading.stack_size(67108864)

67108864

# **Linear programming**

The **linear programming problem** is very common in economics, medicine and other applied sciences, where it is crucial to work within a set of linear models when optimizing one linear form. There are quite a lot of interpretations, but in general all of them in one way or another come to the analysis of the matrix of restrictions, which can be specified by a system of inequalities and equations.


In parallel reality the world is ruled by cats instead of human species and the methods of maximization (or minimization, which are essentially not much different from each other) are working on linear functions, but unfortunately this is not the case! Therefore, the task comes down to carefully constructing a set of feasible solutions and iterating through them in the direction of increasing gradient (which in the case of linear function is just one vector).

---

One thing to add here is that we will be working in $R^n_+$ space, since all the components in our solution are in fact non-negative values (due to specification in applied fields).


We can present the problem in standart form without losing generality!

So the linear function would look like:
$$\sum\limits_{j=1}^n c_jx_j \rightarrow min (max)$$
And the limiting inequalities would tranform from
$\sum\limits_{i=1}^m a_{ij}x_j \leq b_i$
to a rather simple concept:
$$x_i'=b_i-\sum\limits_{i=1}^m a_{ij}x_j \geq 0$$

No matter how much we would like to say that only pseudocompact (closed and bounded sets) will be obtained in the process of solving restrictions systems, this is not entirely true!

The theory clearly states that there is a closed space such that each internal point can be represented in one way as a linear combination of all corner points (a corner point is the point that can't be an internal point of some closed subspace), so:
$$\vec{x}=\sum\limits_{i=1}^k\alpha_i\vec{x_i} \quad \Big| \quad\forall\alpha_i \geq0:\quad\sum\limits_{i=1}^k \alpha_i = 1$$

Thus, if we were to iterate over a **limited** set of corner points, we were destined to find one or an infinite number of points that maximized our linear function.

---

**(Th.)** show that our optimal point should be one in a corner points subset.

$\bullet$ prove by contradiction, so let's assume that there is an optimal point $x^{(0)}$ that does not coincide with the corner one and in terms of maximizing $\forall x \in E:f(x^{(0)}) \geq f(x)$,
well let's look closely on  that **internal** point:
$$f(x^{(0)})=\sum\limits_{i=1}^k \alpha_if(\vec{x_i}) \leq max(f(\vec{x_i})) \sum\limits_{i=1}^k = f(x_k)$$
but that is contradiction!

$q(x)=c_{1}x_1+c_{2}x_2\rightarrow max$

$a_{11}x_1+a_{12}x_2=b_1$

$a_{21}x1+a_{22}x_2\leq b_2$

$a_{31}x1+a_{32}x_2\geq b_3$


|element|coeff|$-x1$    |$-x2$    |$b$  |
|:-----:|:---:|:-------:|:-------:|:---:|
|$x3$   |0    |$a_{11}$ |$a_{12}$ |$b_1$|
|$x4$   |1    |$a_{21}$ |$a_{22}$ |$b_2$|
|$x5$   |1    |$-a_{31}$|$-a_{32}$|$-b_3$|
|$q(x)$ |2    |$-c_{1}$ |$-c_{1}$ |0    |

## **0. Functional and helpers**

In [64]:
@dataclass()
class LinearConstraint:
    coefs: list[float] = field(default_factory=list)
    ctype: str = field(default_factory=lambda: "")
    b: float = 0.0
    is_term_added: bool = False

    def __post_init__(self) -> None:
        self.add_terms()

    def add_terms(self) -> None:
        match self.ctype:
            case 'eq':
                pass
            case 'gte':
                self.coefs = [-coef for coef in self.coefs]
                self.b *= -1
                self.is_term_added = True
            case 'lte':
                self.is_term_added = True
            case _:
                raise ValueError(f'Error: unable to parse mode {self.ctype}')

In [65]:
@dataclass()
class Solution:
    matrix: nptp.NDArray
    basis: nptp.NDArray
    columns: nptp.NDArray

    def __init__(self, file_path: str):
        self.matrix = self.parse(self.validate_path(file_path))
        shape = self.matrix.shape
        self.basis, self.columns = np.arange(shape[1] - 1, shape[1] + shape[0] - 2), np.arange(shape[1] - 1)

    @staticmethod
    def parse(file_path: str) -> nptp.NDArray:
        f_coefs, constraints, goal = [], [], ""
        with open(file_path, "r") as file_:
            data = json.loads(file_.read())
            goal = Solution.validate_goal(data['goal'])
            f_coefs = [-coef if goal == 'max' else coef for coef in data['f']]
            for constraint in data['constraints']:
                constraints.append(
                    LinearConstraint(
                        coefs=constraint['coefs'],
                        ctype=constraint['type'],
                        b=constraint['b']
                        )
                    )
        return Solution.get_simplex_matrix(constraints, f_coefs)

    @staticmethod
    def get_simplex_matrix(constraints: list[LinearConstraint], f_coefs: list[float | int]) -> nptp.NDArray:
        raw_matrix = []
        for constraint in constraints:
            if constraint.is_term_added:
                raw_matrix.append([1.0] + constraint.coefs + [constraint.b])
            else:
                raw_matrix.append([1.0] + constraint.coefs + [constraint.b])
                raw_matrix.append([1.0] + [-coef for coef in constraint.coefs] + [-constraint.b])
        raw_matrix.append([2.0] + f_coefs + [0.0])
        return Solution.sort_matrix(raw_matrix)

    @staticmethod
    def sort_matrix(unsorted_matrix: list[list[float | int]]) -> nptp.NDArray:
        return np.array(
            sorted(unsorted_matrix, key=lambda x: x[0]),
            dtype=np.float64
            )

    @staticmethod
    def validate_path(path: str) -> str:
        if not pathlib.Path(path).is_file():
            raise ValueError(f'Error: no file with path {path}')
        return path

    @staticmethod
    def validate_goal(goal: str) -> str:
        if goal not in ('max', 'min'):
            raise ValueError(f'Error: unable to parse mode {goal}')
        return goal

    def solve(self, *, verbose: bool = False, num_iter: int = 500) -> nptp.NDArray | str | None:
        self.get_initial_plan(num_iter=num_iter)
        return self.get_optimal_plan(verbose=verbose, num_iter=num_iter)

    def get_initial_plan(self, num_iter) -> None:
        log_num_iter = num_iter
        while any(self.matrix[:-1, -1] < 0) and num_iter > 0:
            num_iter -= 1
            i = int(np.argmax(self.matrix[:-1, -1] < 0))
            if all(self.matrix[i, 1:-1] >= 0):
                raise ValueError(f'Error: could not get initial plan for {self.matrix}')
            l = int(np.argmax(self.matrix[i, :-1] < 0))
            old_matrix = self.matrix
            self.transform_jordan(
                *self.get_resolving(
                    initial_row=i,
                    initial_column=l
                    )
                )
        if num_iter == 0:
            warnings.warn(f'Could not get initial plan in {log_num_iter} iterations')

    def get_optimal_plan(self, verbose: bool, num_iter: int) -> nptp.NDArray | str | None:
        log_num_iter = num_iter
        while any(self.matrix[-1, 1:-1] < 0) and num_iter > 0:
            num_iter -= 1
            l = int(np.argmax(self.matrix[-1, 1:-1] < 0)) + 1
            if all(self.matrix[:-1, l] <= 0):
                if verbose:
                    warnings.warn('Linear function is unbound')
                return self.get_current_solution(verbose=verbose)
            i = int(np.argmax(self.matrix[:-1, l] > 0))
            self.transform_jordan(
                *self.get_resolving(
                    initial_row=i,
                    initial_column=l
                    )
                )
        if num_iter == 0:
             warnings.warn(f'Could not converge to optimal plan in {log_num_iter} iterations')

    def get_resolving(self, initial_row: int, initial_column: int) -> tuple[int, int]:
        row, column = initial_row, initial_column
        min_ratio = float('inf')
        for r in range(len(self.matrix) - 1):
            if self.matrix[r][column] != 0:
                ratio = self.matrix[r][-1] / self.matrix[r][column]
                if 0 < ratio < min_ratio:
                    min_ratio = ratio
                    row = r
        return row, column

    def transform_jordan(self, row: int, column: int) -> None:
        new_matrix = copy.copy(self.matrix)
        kernel = self.matrix[row][column]
        self.columns[column], self.basis[row] = self.basis[row], self.columns[column]
        shape_ = self.matrix.shape
        for i in range(shape_[0]):
            for j in range(shape_[1]):
                if i == row and j == column:
                    new_matrix[i][j] = 1 / kernel
                elif j == column:
                    new_matrix[i][j] = -self.matrix[i][j] / kernel
                elif i == row:
                    new_matrix[i][j] = self.matrix[i][j] / kernel
                else:
                    new_matrix[i][j] = self.matrix[i][j] - self.matrix[row][j] * self.matrix[i][column] / kernel
        self.matrix = new_matrix

    def get_current_solution(self, *, verbose: bool = False) -> nptp.NDArray | str:
        n = len(self.basis) + len(self.columns) - 1
        result = np.array([0.0 for _ in range(n)])
        for base, value in zip(self.basis, self.matrix[:-1, -1]):
            result[base - 1] = value
        if verbose:
            return f'Current solution: {result}\nLinear form value: {abs(self.matrix[-1, -1])}\n'
        return result

    def __str__(self):
        table = prettytable.PrettyTable(
            field_names=[f'x{i}' for i in self.columns] + ['b'],
        )
        # table.set_style(prettytable.MARKDOWN)
        table.add_rows(
            [[f'x{j}'] + self.matrix[i, 1:].tolist() for i, j in enumerate(self.basis)],
        )
        table.add_row(['Q'] + self.matrix[-1, 1:].tolist())
        return table.get_string()

## **1. Testing and benchmarking**

In [66]:
@dataclass()
class LinearProgrammingCases:
    json_: dict
    result: nptp.NDArray

In [67]:
TEST_CASES = [
    LinearProgrammingCases(
        json_={
            "f": [1, 2],
            "goal": "max",
            "constraints": [{"coefs": [-3, 2], "type": "lte", "b": 4}, {"coefs": [1, 5], "type": "lte", "b": 20}, {"coefs": [-4, 13], "type": "gte", "b": 30}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [1],
            "goal": "max",
            "constraints": [{"coefs": [2], "type": "lte", "b": 4}, {"coefs": [1], "type": "gte", "b": -3}, {"coefs": [-1], "type": "gte", "b": 4}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [1],
            "goal": "max",
            "constraints": [{"coefs": [1], "type": "gte", "b": -3}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [-6, -1, -4, 5],
            "goal": "min",
            "constraints": [{"coefs": [3, 1, -1, 1], "type": "eq", "b": 4}, {"coefs": [5, 1, 1, -1], "type": "eq", "b": 4}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [-1, -2, -3, 1],
            "goal": "min",
            "constraints": [{"coefs": [1, -3, -1, -2], "type": "eq", "b": -4}, {"coefs": [1, -1, 1, 0], "type": "eq", "b": 0}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [-1, -2, -1, 3, -1],
            "goal": "min",
            "constraints": [{"coefs": [1, 1, 0, 2, 1], "type": "eq", "b": 5}, {"coefs": [1, 1, 1, 3, 2], "type": "eq", "b": 9}, {"coefs": [0, 1, 1, 2, 1], "type": "eq", "b": 6}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [-1, -1, -1, 1, -1],
            "goal": "min",
            "constraints": [{"coefs": [1, 1, 2, 0, 0], "type": "eq", "b": 4}, {"coefs": [0, -2, -2, 1, -1], "type": "eq", "b": -6}, {"coefs": [1, -1, 6, 1, 1], "type": "eq", "b": 12}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [-1, 4, -3, 10],
            "goal": "min",
            "constraints": [{"coefs": [1, 1, -1, -10], "type": "eq", "b": 0}, {"coefs": [1, 14, 10, -10], "type": "eq", "b": 11}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [-1, 5, 1, -1],
            "goal": "min",
            "constraints": [{"coefs": [1, 3, 3, 1], "type": "lte", "b": 3}, {"coefs": [2, 0, 3, -1], "type": "lte", "b": 4}]
            },
        result=np.array([])
        ),
    LinearProgrammingCases(
        json_={
            "f": [1, -1, 1, -1, 2],
            "goal": "min",
            "constraints": [{"coefs": [3, 1, 1, 1, -2], "type": "eq", "b": 10}, {"coefs": [6, 1, 2, 3, -4], "type": "eq", "b": 20}, {"coefs": [10, 1, 3, 6, -7], "type": "eq", "b": 30}]
            },
        result=np.array([])
        )
]

In [68]:
for id_test, test in enumerate(TEST_CASES):
    with open("./tmp.json", "w+") as tmp:
        tmp.write(str(test.json_).replace("'", '"'))
    try:
        s = Solution("./tmp.json")
        print(f'START-TEST{id_test}'.center(80, '~'))
        print(s, end='\n')
        s.solve()
        print(s, end='\n')
        print(s.get_current_solution(verbose=True))
        print(f'END-TEST{id_test}'.center(80, '~'), end='\n\n\n\n')
    except Exception as e:
        print(f'Catched exception on TEST-{id_test}:\n{e}\n')


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~START-TEST0~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
+----+------+-------+-------+
| x0 |  x1  |   x2  |   b   |
+----+------+-------+-------+
| x3 | -3.0 |  2.0  |  4.0  |
| x4 | 1.0  |  5.0  |  20.0 |
| x5 | 4.0  | -13.0 | -30.0 |
| Q  | -1.0 |  -2.0 |  0.0  |
+----+------+-------+-------+
+----+---------------------+----------------------+--------------------+
| x0 |          x4         |          x5          |         b          |
+----+---------------------+----------------------+--------------------+
| x2 | 0.12121212121212122 | -0.03030303030303031 | 3.3333333333333335 |
| x3 |  0.9393939393939394 |  0.5151515151515151  | 7.333333333333334  |
| x1 | 0.39393939393939403 | 0.15151515151515152  | 3.333333333333334  |
| Q  |  0.6363636363636365 | 0.09090909090909093  | 10.000000000000002 |
+----+---------------------+----------------------+--------------------+
Current solution: [3.33333333 3.33333333 7.33333333 0.         0.        ]
Linear form value: 1

<ipython-input-65-5b178ee2924b>:80: UserWarning: Could not get initial plan in 500 iterations
  warnings.warn(f'Could not get initial plan in {log_num_iter} iterations')
